In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microsoft-pens-personalized-news-headlines/final_synthetic.csv
/kaggle/input/microsoft-pens-personalized-news-headlines/Synthetic/Runs_Train.xlsx
/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/personalized_test.tsv
/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/train.tsv
/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/news.tsv
/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/valid.tsv
/kaggle/input/microsoft-pens-personalized-news-headlines/personalization/pers_preprocessed.csv


In [2]:
# dataset_path = '/kaggle/input/microsoft-pens-personalized-news-headlines'
# news_df = pd.read_csv('/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/news.tsv')
# train_df = pd.read_csv('/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/train.tsv')
# valid_df = pd.read_csv('/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/valid.tsv')
dataset_path = '/kaggle/input/microsoft-pens-personalized-news-headlines/PENS/'

# Load news.tsv with tab separator and error handling
try:
    news_df = pd.read_csv(dataset_path + 'news.tsv', sep='\t', quoting=3, on_bad_lines='warn')
    print(news_df.head())
except Exception as e:
    print(f"Error loading file: {e}")

# Check the number of columns
print(f"Number of columns: {len(news_df.columns)}")
print(f"Column names: {news_df.columns.tolist()}")

/tmp/ipykernel_35/3279732354.py:9: ParserWarning: Skipping line 37144: expected 7 fields, saw 9
Skipping line 64363: expected 7 fields, saw 9

  news_df = pd.read_csv(dataset_path + 'news.tsv', sep='\t', quoting=3, on_bad_lines='warn')


  News ID Category         Topic  \
0  N10000   sports        soccer   
1  N10001     news  newspolitics   
2  N10002     news        newsus   
3  N10003     news  newspolitics   
4  N10004     news     newsworld   

                                            Headline  \
0  Predicting Atlanta United's lineup against Col...   
1  Mitch McConnell: DC statehood push is 'full bo...   
2            Home In North Highlands Damaged By Fire   
3  Meghan McCain blames 'liberal media' and 'thir...   
4                            Today in History: Aug 1   

                                           News body  \
0  Only FIVE internationals allowed, count em, FI...   
1  "WASHINGTON -- Senate Majority Leader Mitch Mc...   
2  NORTH HIGHLANDS (CBS13)   Fire damaged a home ...   
3  Meghan McCain is speaking out after a journali...   
4  "1714: George I becomes King Georg Ludwig, Ele...   

                                    Title entity  \
0  "{""Atlanta United's"": 'Atlanta United FC'}"   
1    

In [3]:
# with open(dataset_path + 'news.tsv', 'r') as f:
#     lines = f.readlines()
#     for line_num in [37144, 64363]:
#         line = lines[line_num - 1]
#         tab_count = line.count('\t')
#         print(f"Line {line_num} has {tab_count} tabs, resulting in {tab_count + 1} fields")
#         print(f"Content: {line.strip()}")

In [4]:
train_df = pd.read_csv(dataset_path + 'train.tsv', sep='\t', quoting=3, on_bad_lines='warn')
train_df = train_df.head(100000)
print(len(train_df))
print(train_df.head())

100000
    UserID                                        ClicknewsID  \
0  U335175  N41340 N27570 N83288 N100653 N105490 N88971 N5...   
1  U146053  N95202 N84319 N43563 N78688 N94858 N60541 N109...   
2  U158889  N104156 N103864 N51305 N109347 N70019 N106791 ...   
3   U22232  N25386 N35729 N32113 N102243 N62352 N22227 N10...   
4   U32515  N55509 N15992 N27093 N78610 N13691 N103166 N11...   

                                           dwelltime  \
0  116 23 59 146 661 303 59 199 165 144 110 29 32...   
1  105 62 41 11 52 300 55 17 73 52 145 24 96 38 3...   
2                             175 58 2 871 169 69 10   
3  4 15 3 31 9 26 216 208 58 7 481 32 5 29 13 2 1...   
4  47 123 54 166 29 56 10 131 298 1592 146 48 45 ...   

                                       exposure_time  \
0  6/19/2019 5:10:01 AM#TAB#6/19/2019 5:11:58 AM#...   
1  6/14/2019 11:54:18 AM#TAB#6/14/2019 11:57:02 A...   
2  6/17/2019 1:23:34 PM#TAB#6/17/2019 1:27:47 PM#...   
3  6/15/2019 10:42:34 AM#TAB#6/15/2019 10

In [5]:
valid_df = pd.read_csv(dataset_path + 'valid.tsv', sep='\t', quoting=3, on_bad_lines='warn')
# valid_df.head(1000)
print(valid_df.head())

    UserID                                        ClicknewsID  \
0  U180794  N27499 N49500 N76930 N121334 N117744 N95202 N3...   
1  U311381  N96591 N54516 N39614 N84898 N102774 N10817 N88...   
2   U84844  N90215 N109222 N42583 N59544 N54907 N28628 N99...   
3  U174959  N21530 N14869 N43119 N94858 N91626 N36905 N881...   
4   U56319  N78421 N59628 N43119 N94858 N47462 N49906 N115...   

                                           dwelltime  \
0  59 701 12 73 83 29 1 9 10 15 41 89 18 7 54 163...   
1            133 125 63 13 163 36 77 1207 590 422 70   
2  345 633 521 52 39 116 168 149 159 210 78 17 27...   
3  3 105 28 91 89 6 9 164 605 4 53 284 38 29 10 1...   
4  8 798 706 1186 38 36 54 234 149 71 143 298 121...   

                                       exposure_time  \
0  6/14/2019 6:00:54 AM#TAB#6/14/2019 6:01:54 AM#...   
1  6/17/2019 9:14:48 AM#TAB#6/19/2019 9:07:18 AM#...   
2  6/17/2019 9:19:53 AM#TAB#6/17/2019 1:31:45 PM#...   
3  6/14/2019 3:09:00 PM#TAB#6/14/2019 3:09:12 PM

In [6]:
print("Train DataFrame:")
print("Train columns:", train_df.columns.tolist())
print("\nValidation DataFrame:")
print("Valid columns:", valid_df.columns.tolist())

Train DataFrame:
Train columns: ['UserID', 'ClicknewsID', 'dwelltime', 'exposure_time', 'pos', 'neg', 'start', 'end', 'dwelltime_pos']

Validation DataFrame:
Valid columns: ['UserID', 'ClicknewsID', 'dwelltime', 'exposure_time', 'pos', 'neg', 'start', 'end', 'dwelltime_pos']


In [7]:
print("Sample of train.tsv:")
print(train_df[['UserID', 'ClicknewsID', 'exposure_time', 'start', 'end']].head())
print("\nTimestamp ranges:")
print(f"exposure_time: {train_df['exposure_time'].min()} to {train_df['exposure_time'].max()}")
print(f"start: {train_df['start'].min()} to {train_df['start'].max()}")
print(f"end: {train_df['end'].min()} to {train_df['end'].max()}")

Sample of train.tsv:
    UserID                                        ClicknewsID  \
0  U335175  N41340 N27570 N83288 N100653 N105490 N88971 N5...   
1  U146053  N95202 N84319 N43563 N78688 N94858 N60541 N109...   
2  U158889  N104156 N103864 N51305 N109347 N70019 N106791 ...   
3   U22232  N25386 N35729 N32113 N102243 N62352 N22227 N10...   
4   U32515  N55509 N15992 N27093 N78610 N13691 N103166 N11...   

                                       exposure_time                  start  \
0  6/19/2019 5:10:01 AM#TAB#6/19/2019 5:11:58 AM#...    7/3/2019 6:43:49 AM   
1  6/14/2019 11:54:18 AM#TAB#6/14/2019 11:57:02 A...  6/23/2019 10:28:02 AM   
2  6/17/2019 1:23:34 PM#TAB#6/17/2019 1:27:47 PM#...   6/28/2019 8:50:00 AM   
3  6/15/2019 10:42:34 AM#TAB#6/15/2019 10:43:35 A...    7/3/2019 2:27:50 PM   
4  6/14/2019 3:15:04 PM#TAB#6/14/2019 3:16:31 PM#...   6/26/2019 9:20:16 AM   

                     end  
0    7/3/2019 7:06:06 AM  
1  6/23/2019 10:54:50 AM  
2   6/28/2019 9:00:09 AM  
3    

Splitting training set into train and validation using a Time-based Split

In [8]:
import pandas as pd
from tqdm import tqdm

def temporal_split(data, split_time=None):
    data['first_exposure'] = data['exposure_time'].apply(
        lambda x: pd.to_datetime(x.split('#TAB#')[0], format='%m/%d/%Y %I:%M:%S %p', errors='coerce') if pd.notna(x) else pd.NaT
    )
    
    if split_time is None:
        split_time = data['first_exposure'].quantile(0.8)
    else:
        split_time = pd.to_datetime(split_time)
    
    print(f"Split time: {split_time}")
    
    train_rows = []
    valid_rows = []
    skipped_rows = 0
    
    for _, row in tqdm(data.iterrows(), total=len(data)):
        user_id = row['UserID']
        try:
            exposure_times = row['exposure_time'].split('#TAB#')
            exposure_times = [pd.to_datetime(t, format='%m/%d/%Y %I:%M:%S %p', errors='coerce') for t in exposure_times]
        except:
            skipped_rows += 1
            continue
        
        click_ids = row['ClicknewsID'].split() if pd.notna(row['ClicknewsID']) else []
        pos_ids = row['pos'].split() if pd.notna(row['pos']) else []
        neg_ids = row['neg'].split() if pd.notna(row['neg']) else []
        
        for cid, t in zip(click_ids, exposure_times[:len(click_ids)]):
            if pd.isna(t):
                skipped_rows += 1
                continue
            row_data = (user_id, cid, t, 1)
            if t < split_time:
                train_rows.append(row_data)
            else:
                valid_rows.append(row_data)
        
        t = exposure_times[0] if exposure_times else pd.NaT
        if not pd.isna(t):
            for pid in pos_ids:
                row_data = (user_id, pid, t, 1)
                if t < split_time:
                    train_rows.append(row_data)
                else:
                    valid_rows.append(row_data)
        
        for nid in neg_ids:
            if pd.isna(t):
                skipped_rows += 1
                continue
            row_data = (user_id, nid, t, 0)
            if t < split_time:
                train_rows.append(row_data)
            else:
                valid_rows.append(row_data)
    
    print(f"Skipped {skipped_rows} rows due to invalid timestamps")
    
    columns = ['UserID', 'NewsID', 'exposure_time', 'Clicked']
    train_interactions_df = pd.DataFrame(train_rows, columns=columns)
    valid_interactions_df = pd.DataFrame(valid_rows, columns=columns)
    
    train_interactions_df = train_interactions_df.drop_duplicates(subset=['UserID', 'NewsID'])
    valid_interactions_df = valid_interactions_df.drop_duplicates(subset=['UserID', 'NewsID'])
    
    train_interactions_df.to_csv('/kaggle/working/train_split.csv', index=False)
    valid_interactions_df.to_csv('/kaggle/working/valid_split.csv', index=False)
    
    print(f"Validation interactions: {len(train_interactions_df)}")
    print(f"Test interactions: {len(valid_interactions_df)}")
    print(f"Validation positive: {len(train_interactions_df[train_interactions_df['Clicked'] == 1])}")
    print(f"Test positive: {len(valid_interactions_df[valid_interactions_df['Clicked'] == 1])}")
    print(f"Validation negative: {len(train_interactions_df[train_interactions_df['Clicked'] == 0])}")
    print(f"Test negative: {len(valid_interactions_df[valid_interactions_df['Clicked'] == 0])}")
    return train_interactions_df, valid_interactions_df


train_interactions_df, valid_interactions_df = temporal_split(valid_df)

Split time: 2019-06-17 09:20:16.800000


100%|██████████| 100000/100000 [23:10<00:00, 71.89it/s] 


Skipped 0 rows due to invalid timestamps
Validation interactions: 3696091
Test interactions: 10762218
Validation positive: 1367150
Test positive: 10185762
Validation negative: 2328941
Test negative: 576456


In [9]:
# def extract_interactions(df):
#     interactions = []
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         user_id = row['UserID']
#         click_ids = row['ClicknewsID'].split() if pd.notna(row['ClicknewsID']) else []
#         for cid in click_ids:
#             interactions.append((user_id, cid, 1))
#         if pd.notna(row['pos']):
#             for news_id in row['pos'].split():
#                 interactions.append((user_id, news_id, 1))
#         if pd.notna(row['neg']):
#             for news_id in row['neg'].split():
#                 interactions.append((user_id, news_id, 0))
#     return pd.DataFrame(interactions, columns=['UserID', 'NewsID', 'Clicked'])

# train_df = pd.read_csv(dataset_path + 'train.tsv', sep='\t', quoting=3, on_bad_lines='warn', nrows=400000)
# train_interactions_df = extract_interactions(train_df)
# train_interactions_df = train_interactions_df.drop_duplicates(subset=['UserID', 'NewsID'])
# train_interactions_df.to_csv('/kaggle/working/train_interactions.csv', index=False)

# print(f"Training interactions: {len(train_interactions_df)}")
# print(f"Training positive: {len(train_interactions_df[train_interactions_df['Clicked'] == 1])}")
# print(f"Training negative: {len(train_interactions_df[train_interactions_df['Clicked'] == 0])}")
# print("Training sample:")
# print(train_interactions_df.head())

In [10]:
from scipy.sparse import csr_matrix
import numpy as np

user_ids = train_interactions_df['UserID'].unique()
news_ids = train_interactions_df['NewsID'].unique()
user2idx = {uid: idx for idx, uid in enumerate(user_ids)}
news2idx = {nid: idx for idx, nid in enumerate(news_ids)}

rows, cols, data = [], [], []
for _, row in train_interactions_df.iterrows():
    rows.append(user2idx[row['UserID']])
    cols.append(news2idx[row['NewsID']])
    data.append(row['Clicked'])

interaction_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(news_ids)))
print(f"Interaction matrix shape: {interaction_matrix.shape}")
print(f"Number of non-zero entries: {interaction_matrix.nnz}")

Interaction matrix shape: (77924, 20271)
Number of non-zero entries: 3696091


In [11]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.7 MB/s eta 0:00:00:00:010:01


In [12]:
from implicit.als import AlternatingLeastSquares

# Train ALS model
model = AlternatingLeastSquares(factors=50, iterations=15, regularization=0.1)
model.fit(interaction_matrix)  # Users as rows

def recommend_items(user_id, interaction_matrix, model, user2idx, news2idx, top_n=10):
    if user_id not in user2idx:
        return []
    user_idx = user2idx[user_id]
    user_items = interaction_matrix[user_idx:user_idx+1]
    rec_indices, _ = model.recommend(user_idx, user_items, N=top_n, filter_already_liked_items=True)
    top_news_ids = [list(news2idx.keys())[list(news2idx.values()).index(idx)] for idx in rec_indices]
    return top_news_ids

sample_user = train_interactions_df['UserID'].iloc[0]
recommendations = recommend_items(sample_user, interaction_matrix, model, user2idx, news2idx)
print(f"Recommendations for user {sample_user}: {recommendations}")

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

Recommendations for user U180794: ['N121349', 'N39862', 'N81795', 'N75910', 'N122875', 'N31171', 'N55111', 'N51924', 'N119816', 'N116986']


In [16]:
import numpy as np
from tqdm.notebook import tqdm

def evaluate_recommendations(eval_df, interaction_matrix, model_or_similarity, user2idx, news2idx, top_n=10, use_implicit=False, sample_frac=1.0):
    recall_sum, precision_sum, ndcg_sum, mrr_sum, total = 0, 0, 0, 0, 0
    
    # Get unique users and sample if sample_frac < 1.0
    unique_users = eval_df['UserID'].unique()
    if sample_frac < 1.0:
        unique_users = np.random.choice(unique_users, size=int(len(unique_users) * sample_frac), replace=False)
        print(f"Evaluating on {len(unique_users)} sampled users ({sample_frac*100:.1f}% of {len(eval_df['UserID'].unique())} total users)")
    
    # Evaluate with tqdm progress bar
    for user_id in tqdm(unique_users, desc="Evaluating users", leave=True):
        if user_id not in user2idx:
            continue
        # Get ground truth (positive interactions)
        ground_truth = set(eval_df[(eval_df['UserID'] == user_id) & (eval_df['Clicked'] == 1)]['NewsID'].tolist())
        if not ground_truth:
            continue
            
        # Get recommendations
        if use_implicit:
            recs = recommend_items(user_id, interaction_matrix, model_or_similarity, user2idx, news2idx, top_n)
        else:
            recs = recommend_items(user_id, interaction_matrix, model_or_similarity, news2idx, top_n)
        recs = set(recs)
        
        # Relevant items in recommendations
        relevant = len(recs & ground_truth)
        
        # Recall@K: Proportion of relevant items retrieved
        recall = relevant / len(ground_truth) if len(ground_truth) > 0 else 0
        recall_sum += recall
        
        # Precision@K: Proportion of recommended items that are relevant
        precision = relevant / len(recs) if len(recs) > 0 else 0
        precision_sum += precision
        
        # NDCG@K
        dcg = 0
        for i, item in enumerate(recs, 1):
            if item in ground_truth:
                dcg += 1 / np.log2(i + 1)
        idcg = sum(1 / np.log2(i + 1) for i in range(1, min(len(ground_truth), top_n) + 1))
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_sum += ndcg
        
        # MRR: Reciprocal of the rank of the first relevant item
        for rank, item in enumerate(recs, 1):
            if item in ground_truth:
                mrr_sum += 1 / rank
                break
        
        total += 1
    
    recall_k = recall_sum / total if total > 0 else 0
    precision_k = precision_sum / total if total > 0 else 0
    ndcg_k = ndcg_sum / total if total > 0 else 0
    mrr = mrr_sum / total if total > 0 else 0
    
    return recall_k, precision_k, ndcg_k, mrr

# Evaluate on validation set (25% sample)
try:
    # Using implicit.als
    recall_valid, precision_valid, ndcg_valid, mrr_valid = evaluate_recommendations(
        valid_interactions_df, interaction_matrix, model, user2idx, news2idx, use_implicit=True, sample_frac=0.01
    )
except:
    # Fallback to cosine_similarity
    recall_valid, precision_valid, ndcg_valid, mrr_valid = evaluate_recommendations(
        valid_interactions_df, interaction_matrix, item_similarity, user2idx, news2idx, sample_frac=0.25
    )
print(f"Validation Recall@10: {recall_valid:.4f}, Precision@10: {precision_valid:.4f}, NDCG@10: {ndcg_valid:.4f}, MRR: {mrr_valid:.4f}")

Evaluating on 974 sampled users (1.0% of 97437 total users)


Evaluating users:   0%|          | 0/974 [00:00<?, ?it/s]

Validation Recall@10: 0.0011, Precision@10: 0.0124, NDCG@10: 0.0144, MRR: 0.0453
